<a href="https://colab.research.google.com/github/heydefranca/CancerTypePrediction/blob/master/C%C3%B3pia_de_MIRNA_vers%C3%A3o_1D_e_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1D

In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import optimizers, regularizers
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Dense, Dropout
from keras.layers.convolutional import Conv1D, Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalAveragePooling2D, GlobalAveragePooling1D
from keras.layers import Input, Concatenate, add
from keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import metrics
drive.mount('/content/drive')

Mounted at /content/drive


### Lendo e tratando os dados concatenados de X

In [2]:
concatx_url = '/content/drive/MyDrive/Doutorado/concatx.csv'
df_x = pd.read_csv(concatx_url, sep=";", header=None)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3763) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_x.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3724,3725,3726,3727,3728,3729,3730,3731,3732,3733,3734,3735,3736,3737,3738,3739,3740,3741,3742,3743,3744,3745,3746,3747,3748,3749,3750,3751,3752,3753,3754,3755,3756,3757,3758,3759,3760,3761,3762,3763
0,NaN,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,3723,3724,3725,3726,3727,3728,3729,3730,3731,3732,3733,3734,3735,3736,3737,3738,3739,3740,3741,3742,3743,3744,3745,3746,3747,3748,3749,3750,3751,3752,3753,3754,3755,3756,3757,3758,3759,3760,3761,classe
1,0.0,8735,8442,8801,23959,7706,1538,889,2534,2582,1893,1651,1,4,3483,6100,5875,9755,9944,0,0,0,1,419,5809,222,12931,30635,0,0,64,0,0,0,0,0,0,2,1,0,...,0,0,0,0,0,0,97872,97423,98382,0,0,0,0,64920,62742,10988,128886,0,0,3550,0,0,0,1411,2937,0,0,0,0,0,3820,0,9533,4607,0,8581,8922,49026,155620,1
2,1.0,35173,35187,35476,47475,5295,1503,5116,15442,15725,1824,3027,12,9,8847,22184,22170,74151,74671,0,0,110,120,21,2512,640,11536,45312,0,0,79,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,19401,19246,19495,0,0,0,0,68222,66193,10024,51794,0,0,3070,0,2815,0,0,1603,0,0,0,0,0,2927,0,11422,1309,0,3464,6595,14393,109649,1
3,2.0,32653,32514,32383,29587,9656,2539,2136,25402,25993,2778,2515,30,28,16909,16125,16055,44949,44516,0,0,188,175,11,3341,237,87284,101893,0,0,114,1,0,0,0,0,0,0,0,0,...,0,1166,0,0,0,0,20949,20655,20365,0,0,0,0,76089,73480,7531,74578,0,1428,4283,0,1009,0,1542,3448,0,0,0,0,0,4162,0,17792,2734,0,5769,9451,26362,120333,1
4,3.0,63110,63501,63008,75246,6530,2727,8017,30976,31283,1839,2141,7944,8287,19535,31623,32384,40136,40177,0,0,0,1,25,3441,518,16911,110782,0,1,108,4,0,0,0,0,0,3,1,1,...,0,2418,0,0,0,0,8046,8108,8033,0,0,0,0,58286,56364,12249,45637,0,0,1489,0,3300,0,1420,2154,0,0,0,0,0,2580,0,9505,5389,0,2732,8603,17450,118126,1


In [11]:
df_x = df_x.drop(3763, 1)
df_x = df_x.drop(0, 1)
df_x = df_x.drop(0, 0)

KeyError: ignored

In [5]:
df_x.head(5)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3723,3724,3725,3726,3727,3728,3729,3730,3731,3732,3733,3734,3735,3736,3737,3738,3739,3740,3741,3742,3743,3744,3745,3746,3747,3748,3749,3750,3751,3752,3753,3754,3755,3756,3757,3758,3759,3760,3761,3762
1,8735,8442,8801,23959,7706,1538,889,2534,2582,1893,1651,1,4,3483,6100,5875,9755,9944,0,0,0,1,419,5809,222,12931,30635,0,0,64,0,0,0,0,0,0,2,1,0,0,...,2821,0,0,0,0,0,0,97872,97423,98382,0,0,0,0,64920,62742,10988,128886,0,0,3550,0,0,0,1411,2937,0,0,0,0,0,3820,0,9533,4607,0,8581,8922,49026,155620
2,35173,35187,35476,47475,5295,1503,5116,15442,15725,1824,3027,12,9,8847,22184,22170,74151,74671,0,0,110,120,21,2512,640,11536,45312,0,0,79,0,0,0,0,0,0,1,0,0,3,...,5007,0,0,0,0,0,0,19401,19246,19495,0,0,0,0,68222,66193,10024,51794,0,0,3070,0,2815,0,0,1603,0,0,0,0,0,2927,0,11422,1309,0,3464,6595,14393,109649
3,32653,32514,32383,29587,9656,2539,2136,25402,25993,2778,2515,30,28,16909,16125,16055,44949,44516,0,0,188,175,11,3341,237,87284,101893,0,0,114,1,0,0,0,0,0,0,0,0,0,...,2734,0,1166,0,0,0,0,20949,20655,20365,0,0,0,0,76089,73480,7531,74578,0,1428,4283,0,1009,0,1542,3448,0,0,0,0,0,4162,0,17792,2734,0,5769,9451,26362,120333
4,63110,63501,63008,75246,6530,2727,8017,30976,31283,1839,2141,7944,8287,19535,31623,32384,40136,40177,0,0,0,1,25,3441,518,16911,110782,0,1,108,4,0,0,0,0,0,3,1,1,2,...,5001,0,2418,0,0,0,0,8046,8108,8033,0,0,0,0,58286,56364,12249,45637,0,0,1489,0,3300,0,1420,2154,0,0,0,0,0,2580,0,9505,5389,0,2732,8603,17450,118126
5,40067,39849,40175,44984,2268,2649,5055,23009,23125,2012,3020,1235,1381,7605,7964,8213,104141,103838,0,0,494,510,26,2653,290,20004,97000,0,0,128,1,0,0,0,0,0,1,0,0,0,...,3425,0,0,0,0,0,0,8439,7853,7800,0,0,0,0,99116,93012,9226,120522,0,0,1045,0,3001,0,1045,1732,0,0,0,0,0,6136,0,17332,1477,0,4844,8834,10957,154370


In [6]:
df_x.dtypes

1       int64
2       int64
3       int64
4       int64
5       int64
        ...  
3758    int64
3759    int64
3760    int64
3761    int64
3762    int64
Length: 3762, dtype: object

In [7]:
#df1 = df_x.reindex(columns=['80','173','2384','2750','1688','966','1124','1604','2482','96','2215','1413','2499','3052','1078','181','2276','1333','2146','471'])

In [23]:

import pandas as pd
df1= df_x.iloc[:,[79,172,2383,2749,1687,965,1123,1603,2481,95,2214,1412,2498,3051,1077,180,2275,1332,2145,470]]
#df1= df_x.iloc[:,[80,173,2384,2750,1688,966,1124,1604,2482,96,2215,1413,2499,3052,1078,181,2276,1333,2146,471]]

In [24]:

df1.shape

(552, 20)

In [25]:
df1.head(25)

,80,173,2384,2750,1688,966,1124,1604,2482,96,2215,1413,2499,3052,1078,181,2276,1333,2146,471
1,2,0,0,3643,0,0,4,1,4310,0,0,0,0,0,0,1778,0,0,64376,3996
2,0,0,0,1389,0,0,7,1,1964,0,0,0,0,0,0,72,0,0,39397,2161
3,0,0,0,1166,0,0,2,0,1166,0,0,0,0,0,0,128,0,0,39004,2589
4,0,0,0,1680,0,0,13,1,3111,0,0,0,0,0,0,71,0,0,37977,2814
5,0,0,0,2216,0,0,1,3,1567,0,0,0,0,0,0,125,0,0,39869,4907
6,0,0,0,1376,0,0,3,1,0,0,0,0,0,0,0,151,0,0,33619,1622
7,1,0,0,1519,0,0,2,3,1679,0,0,0,0,0,0,76,0,0,44788,2605
8,0,0,0,2325,0,0,3,1,2420,0,0,0,0,0,0,192,0,0,57905,2709
9,0,0,0,1537,0,0,4,1,1299,0,0,0,0,0,0,218,0,0,28166,3969
10,0,0,0,1100,0,0,5,3,1934,0,0,0,0,0,2,487,0,0,40592,3336


### Lendo e tratando os dados concatenados de Y

In [12]:
concaty_url = '/content/drive/MyDrive/Doutorado/concaty.csv'
df_y = pd.read_csv(concaty_url, sep=";", header=None)

In [13]:
df_y = df_y.drop(0, 1)
df_y = df_y.drop(0, 0)

In [14]:
df_y.head(5)

,1
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0


In [15]:
df_y.shape

(552, 1)

In [16]:
df_y.dtypes

1    float64
dtype: object

In [17]:
df_y= df_y.astype(int)

In [18]:
df_y.dtypes

1    int64
dtype: object

### Separando dados

In [ ]:
df_x= df1
df_x.dtypes
#df_x= df_x.fillna(0)
df_x= df_x.astype(int)

In [ ]:
df_x.dtypes

80      int64
173     int64
2384    int64
2750    int64
1688    int64
966     int64
1124    int64
1604    int64
2482    int64
96      int64
2215    int64
1413    int64
2499    int64
3052    int64
1078    int64
181     int64
2276    int64
1333    int64
2146    int64
471     int64
dtype: object

In [28]:
nonzero_mean = df_x.mean()
nonzero_mean
nonzero_mean1=nonzero_mean.mean()
nonzero_mean1
#df['coloum'] = df['coloum'].replace(['value_1','valu_2'],'new_value')
df_x.replace(0, nonzero_mean1, inplace = True)

In [29]:
df_x

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3723,3724,3725,3726,3727,3728,3729,3730,3731,3732,3733,3734,3735,3736,3737,3738,3739,3740,3741,3742,3743,3744,3745,3746,3747,3748,3749,3750,3751,3752,3753,3754,3755,3756,3757,3758,3759,3760,3761,3762
1,8735,8442,8801,23959,7706,1538,889,2534,2582,1893,1651,1.0,4.0,3483,6100,5875,9755,9944,3201.106778,3201.106778,3201.106778,1.000000,419,5809,222,12931,30635,3201.106778,3201.106778,64,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,2.000000,1.000000,3201.106778,3201.106778,...,2821.0,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,97872,97423,98382,3201.106778,3201.106778,3201.106778,3201.106778,64920,62742,10988,128886,3201.106778,3201.106778,3550.000000,3201.106778,3201.106778,3201.106778,1411.000000,2937.000000,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,3820.000000,3201.106778,9533.0,4607.0,3201.106778,8581.0,8922,49026,155620
2,35173,35187,35476,47475,5295,1503,5116,15442,15725,1824,3027,12.0,9.0,8847,22184,22170,74151,74671,3201.106778,3201.106778,110.000000,120.000000,21,2512,640,11536,45312,3201.106778,3201.106778,79,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,1.000000,3201.106778,3201.106778,3.000000,...,5007.0,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,19401,19246,19495,3201.106778,3201.106778,3201.106778,3201.106778,68222,66193,10024,51794,3201.106778,3201.106778,3070.000000,3201.106778,2815.000000,3201.106778,3201.106778,1603.000000,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,2927.000000,3201.106778,11422.0,1309.0,3201.106778,3464.0,6595,14393,109649
3,32653,32514,32383,29587,9656,2539,2136,25402,25993,2778,2515,30.0,28.0,16909,16125,16055,44949,44516,3201.106778,3201.106778,188.000000,175.000000,11,3341,237,87284,101893,3201.106778,3201.106778,114,1.000000,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,...,2734.0,3201.106778,1166.000000,3201.106778,3201.106778,3201.106778,3201.106778,20949,20655,20365,3201.106778,3201.106778,3201.106778,3201.106778,76089,73480,7531,74578,3201.106778,1428.000000,4283.000000,3201.106778,1009.000000,3201.106778,1542.000000,3448.000000,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,4162.000000,3201.106778,17792.0,2734.0,3201.106778,5769.0,9451,26362,120333
4,63110,63501,63008,75246,6530,2727,8017,30976,31283,1839,2141,7944.0,8287.0,19535,31623,32384,40136,40177,3201.106778,3201.106778,3201.106778,1.000000,25,3441,518,16911,110782,3201.106778,1.000000,108,4.000000,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,3.000000,1.000000,1.000000,2.000000,...,5001.0,3201.106778,2418.000000,3201.106778,3201.106778,3201.106778,3201.106778,8046,8108,8033,3201.106778,3201.106778,3201.106778,3201.106778,58286,56364,12249,45637,3201.106778,3201.106778,1489.000000,3201.106778,3300.000000,3201.106778,1420.000000,2154.000000,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,2580.000000,3201.106778,9505.0,5389.0,3201.106778,2732.0,8603,17450,118126
5,40067,39849,40175,44984,2268,2649,5055,23009,23125,2012,3020,1235.0,1381.0,7605,7964,8213,104141,103838,3201.106778,3201.106778,494.000000,510.000000,26,2653,290,20004,97000,3201.106778,3201.106778,128,1.000000,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,1.000000,3201.106778,3201.106778,3201.106778,...,3425.0,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,8439,7853,7800,3201.106778,3201.106778,3201.106778,3201.106778,99116,93012,9226,120522,3201.106778,3201.106778,1045.000000,3201.106778,3001.000000,3201.106778,1045.000000,1732.000000,3201.106778,3201.106778,3201.106778,3201.106778,3201.106778,6136.000000,3201.106778,17332.0,1477.0,3201.106778,4844.0,8834,10957,154370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [30]:
df_x.dtypes

1         int64
2         int64
3         int64
4         int64
5         int64
         ...   
3758    float64
3759    float64
3760      int64
3761      int64
3762      int64
Length: 3762, dtype: object

In [31]:
df_x= df_x.astype(int)
df_x.head(15)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3723,3724,3725,3726,3727,3728,3729,3730,3731,3732,3733,3734,3735,3736,3737,3738,3739,3740,3741,3742,3743,3744,3745,3746,3747,3748,3749,3750,3751,3752,3753,3754,3755,3756,3757,3758,3759,3760,3761,3762
1,8735,8442,8801,23959,7706,1538,889,2534,2582,1893,1651,1,4,3483,6100,5875,9755,9944,3201,3201,3201,1,419,5809,222,12931,30635,3201,3201,64,3201,3201,3201,3201,3201,3201,2,1,3201,3201,...,2821,3201,3201,3201,3201,3201,3201,97872,97423,98382,3201,3201,3201,3201,64920,62742,10988,128886,3201,3201,3550,3201,3201,3201,1411,2937,3201,3201,3201,3201,3201,3820,3201,9533,4607,3201,8581,8922,49026,155620
2,35173,35187,35476,47475,5295,1503,5116,15442,15725,1824,3027,12,9,8847,22184,22170,74151,74671,3201,3201,110,120,21,2512,640,11536,45312,3201,3201,79,3201,3201,3201,3201,3201,3201,1,3201,3201,3,...,5007,3201,3201,3201,3201,3201,3201,19401,19246,19495,3201,3201,3201,3201,68222,66193,10024,51794,3201,3201,3070,3201,2815,3201,3201,1603,3201,3201,3201,3201,3201,2927,3201,11422,1309,3201,3464,6595,14393,109649
3,32653,32514,32383,29587,9656,2539,2136,25402,25993,2778,2515,30,28,16909,16125,16055,44949,44516,3201,3201,188,175,11,3341,237,87284,101893,3201,3201,114,1,3201,3201,3201,3201,3201,3201,3201,3201,3201,...,2734,3201,1166,3201,3201,3201,3201,20949,20655,20365,3201,3201,3201,3201,76089,73480,7531,74578,3201,1428,4283,3201,1009,3201,1542,3448,3201,3201,3201,3201,3201,4162,3201,17792,2734,3201,5769,9451,26362,120333
4,63110,63501,63008,75246,6530,2727,8017,30976,31283,1839,2141,7944,8287,19535,31623,32384,40136,40177,3201,3201,3201,1,25,3441,518,16911,110782,3201,1,108,4,3201,3201,3201,3201,3201,3,1,1,2,...,5001,3201,2418,3201,3201,3201,3201,8046,8108,8033,3201,3201,3201,3201,58286,56364,12249,45637,3201,3201,1489,3201,3300,3201,1420,2154,3201,3201,3201,3201,3201,2580,3201,9505,5389,3201,2732,8603,17450,118126
5,40067,39849,40175,44984,2268,2649,5055,23009,23125,2012,3020,1235,1381,7605,7964,8213,104141,103838,3201,3201,494,510,26,2653,290,20004,97000,3201,3201,128,1,3201,3201,3201,3201,3201,1,3201,3201,3201,...,3425,3201,3201,3201,3201,3201,3201,8439,7853,7800,3201,3201,3201,3201,99116,93012,9226,120522,3201,3201,1045,3201,3001,3201,1045,1732,3201,3201,3201,3201,3201,6136,3201,17332,1477,3201,4844,8834,10957,154370
6,32482,32903,32895,55166,1611,1842,3363,12311,12342,971,1593,109,81,3911,6164,6309,31297,31296,3201,3201,1,3201,32,2365,220,2377,49264,3201,1,114,1,3201,3201,3201,3201,3201,1,3201,3201,3201,...,2820,3201,2683,3201,3201,3201,3201,58531,58190,58459,3201,3201,3201,3201,143823,141445,17974,86917,3201,1628,1507,3201,7760,3201,1946,3535,3201,3201,3201,3201,3201,3077,3201,18037,2132,3201,4352,7711,11331,128669
7,46022,46006,45928,52039,2581,4829,3043,22482,23364,1447,3094,2,2,6772,7713,7983,73069,73246,3201,3201,7,5,67,1803,229,1253,74217,3201,1,113,3201,3201,3201,3201,3201,3201,2,2,3201,3201,...,3755,3201,3201,3201,3201,3201,3201,128823,129007,129196,3201,3201,3201,3201,165521,158533,10215,149653,3201,3201,1132,3201,3201,3201,1013,1240,3201,3201,3201,3201,3201,4443,3201,37353,1601,3201,5839,7425,10942,104032
8,44832,44466,45042,93954,7679,2669,5079,20776,20825,2053,5353,60,73,16586,12654,12563,43925,44135,3201,3201,37,57,21,2639,249,10176,91901,3201,1,56,3201,3201,3201,3201,3201,3201,1,1,3201,3201,...,4502,3201,1644,3201,3201,3201,3201,10790,11028,11370,3201,3201,3201,3201,71280,69507,9287,56237,3201,1061,1162,3201,1838,3201,1501,3148,3201,3201,3201,3201,3201,2887,3201,18453,2642,3201,3001,9079,17189,132956
9,48068,47712,48587,92745,6956,4488,3108,16760,17166,1797,2186,8,14,15751,25754,26155,67729,67742,3201,3201,235,237,40,3493,460,70459,95235,3201,3201,81,4,3201,3201,3201,3201,3201,3201,3201,3201,3201,...,2875,3201,2532,3201,3201,3201,3201,8673,8526,8634,3201,3201,3201,3201,101326,97367,9155,69282,3201,3201,2359,3201,4442,3201,1162,1006,3201,3201,3201,3201,3201,3260,3201,8456,1926,3201,4385,5571,18922,127459
10,1402

In [32]:
x_training, x_validation, y_training, y_validation = train_test_split(df_x, df_y, test_size=0.1, random_state=5)

In [33]:
x_training.shape

(496, 3762)

### Criando o modelo

In [56]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import mean_squared_error as mse
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import MaxPool1D
from keras.optimizer_v2.adam import Adam as Adam
from keras.models import Sequential, Model
Adam = tf.keras.optimizers.Adam
Adam = tf.keras.optimizers.Adam
inputs = Input(shape=((x_training.shape[1], 1)))

conv1 = Conv1D(8, 3, padding='same', activation='relu')(inputs)
conv2 = Conv1D(16, 3, padding='same', activation='relu')(conv1)
#pool1 = GlobalAveragePooling1D()(conv2)
pool1= MaxPool1D()(conv2)
dense1 = Dense(100, activation='relu')(conv2)
drop1 = Dropout(0.2)(dense1)
dense2 = Dense(100, activation='relu')(drop1)
drop2 = Dropout(0.2)(dense2)
#outputs= XGBClassifier(max_depth=30, learning_rate=0.1, n_estimators=100, verbosity=1, silent=None, objective="binary:logistic", booster='gbtree', n_jobs=1, nthread=None, gamma=0)(drop2)
outputs = Dense(1, activation='softmax')(drop2)

model = tf.keras.Model(inputs, outputs)

# print (model.summary())
opt = SGD(lr=0.01, momentum=0.9)
#sgd = SGD(learning_rate=0.001, nesterov=True);
custom_adam =tf.keras.optimizers.Adam(lr=0.05, decay=0.9, amsgrad=True)
#model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=['acc', 'mae'])
##model.compile(loss=tf.keras.losses.BinaryCrossentropy(reduction='none'), optimizer=custom_adam, metrics=['accuracy'])
#model.compile(loss='mean_absolute_error', optimizer=sgd)
#model.compile(loss='mse', optimizer='adam', metrics=['acc', 'mae'])
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")
model_dense2_output = Model(inputs=inputs, outputs=dense2)



adam = Adam(lr=1e-4)

# We add metrics to get more results you want to see
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

print('Training ------------')
# Another way to train the model
model.fit(x_training, y_training, epochs=1, batch_size=16)

print('\nTesting ------------')
# Evaluate the model with the metrics we defined earlier
loss, accuracy = model.evaluate(x_validation, y_validation)

print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)


#with xgboost
import xgboost
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import GridSearchCV
import xgboost as xgb



data_dmatrix = xgb.DMatrix(df_x,label=df_y)
data_dmatrix
X_train_xg = model_dense2_output.predict(data_dmatrix)
X_test_xg = model_dense2_output.predict(x_validation)
print(np.array(X_train_xg).shape)
print(np.array(y_training).shape)


dataset = xgboost.XGBClassifier(X_train_xg, label=y_training)
watchlist = [(data_dmatrix, 'train')]
params = {'max_depth':7, 'eta':0.1, 'silent':1, 'num_class':1,'objective':'multi:softmax' } 
model_xg = xgboost.train(params, dataset, num_boost_round=150, evals=watchlist)


test_x = xgboost.XGBClassifier(x_validation)
result = model_xg.predict(x_training)
#print(t)
#m = LogisticRegression()
#m = SVC(kernel='rbf',decision_function_shape='ovr')
#m.fit(X,y)
#print(m)

print(metrics.classification_report(y_validation, result))
print(metrics.confusion_matrix(y_validation, result))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier()


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


AttributeError: ignored

In [48]:
# multiclass classification
import pandas
import xgboost
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
model = xgboost.XGBClassifier()
model.fit(x_training, y_training)
print(model)
# make predictions for test data
y_pred = model.predict(x_validation)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_validation, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

print(metrics.classification_report(y_validation, predictions))
print(metrics.confusion_matrix(y_validation, predictions))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier()
Accuracy: 96.43%
              precision    recall  f1-score   support

           0       1.00      0.33      0.50         3
           1       0.96      1.00      0.98        53

    accuracy                           0.96        56
   macro avg       0.98      0.67      0.74        56
weighted avg       0.97      0.96      0.96        56

[[ 1  2]
 [ 0 53]]


### Treinando o modelo

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
model.fit(x_training, y_training, epochs=50, validation_split=0.3, shuffle=True, validation_data=(x_validation,y_validation), callbacks=[EarlyStopping(monitor='val_loss', patience=15)] ) 

Epoch 1/50
11/11 [==============================] - 1s 31ms/step - loss: 1.1942 - accuracy: 0.9222 - val_loss: 1.0386 - val_accuracy: 0.9329
Epoch 2/50
11/11 [==============================] - 0s 9ms/step - loss: 1.1872 - accuracy: 0.9222 - val_loss: 1.0386 - val_accuracy: 0.9329
Epoch 3/50
11/11 [==============================] - 0s 10ms/step - loss: 1.2011 - accuracy: 0.9222 - val_loss: 1.0386 - val_accuracy: 0.9329
Epoch 4/50
11/11 [==============================] - 0s 9ms/step - loss: 1.1872 - accuracy: 0.9222 - val_loss: 1.0386 - val_accuracy: 0.9329
Epoch 5/50
11/11 [==============================] - 0s 9ms/step - loss: 1.1872 - accuracy: 0.9222 - val_loss: 1.0386 - val_accuracy: 0.9329
Epoch 6/50
11/11 [==============================] - 0s 8ms/step - loss: 1.1872 - accuracy: 0.9222 - val_loss: 1.0386 - val_accuracy: 0.9329
Epoch 7/50
11/11 [==============================] - 0s 9ms/step - loss: 1.1594 - accuracy: 0.9222 - val_loss: 1.0386 - val_accuracy: 0.9329
Epoch 8/50
11/11 [

In [ ]:
model.evaluate(x_validation, y_validation)


2/2 [==============================] - 0s 5ms/step - loss: 0.7625 - accuracy: 0.9464


[0.7624620199203491, 0.9464285969734192]

### Realizando predições

In [ ]:
#y_pred = model.predict(x_validation)
y_pred = model.predict(x_training, verbose=0)
y_pred=y_pred.astype(int)
y_pred

array([[[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       ...,

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]]])

In [ ]:
y_pred.shape

(496, 20, 1)

In [ ]:
import pandas as pd
y_pred1= y_pred[:, [0]]

In [ ]:
y_pred1.shape

(496, 1, 1)

In [ ]:
ypred=y_pred1[:,0]

In [ ]:
y_pred1= y_pred[:, [0]]

In [ ]:
def column(matrix, i):
    return [row[i] for row in matrix]

In [ ]:
yp=column(y_pred1, 0)


In [ ]:
ypred= pd.DataFrame(yp)

In [ ]:
ypred.shape

(496, 1)

In [ ]:
ypred.dtypes

0    int64
dtype: object

In [ ]:
y_training.shape

(496, 1)

In [ ]:
y_training.dtypes

1    int64
dtype: object

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(y_training, ypred)
print (classification_report(y_training, ypred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        37
           1       0.93      1.00      0.96       459

    accuracy                           0.93       496
   macro avg       0.46      0.50      0.48       496
weighted avg       0.86      0.93      0.89       496



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Avaliando métricas

In [ ]:
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 20, 1)]           0         
                                                                 
 conv1d_16 (Conv1D)          (None, 20, 8)             32        
                                                                 
 conv1d_17 (Conv1D)          (None, 20, 16)            400       
                                                                 
 dense_24 (Dense)            (None, 20, 100)           1700      
                                                                 
 dropout_16 (Dropout)        (None, 20, 100)           0         
                                                                 
 dense_25 (Dense)            (None, 20, 100)           10100     
                                                                 
 dropout_17 (Dropout)        (None, 20, 100)           0   

Matriz de confusão

In [ ]:
confusion_matrix(y_training, ypred)

array([[  0,  37],
       [  0, 459]])

Precisão

In [ ]:
precision_score(y_training, ypred, average='macro')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.4627016129032258

Recall

In [ ]:
recall_score(y_training, ypred, average='macro')

0.5

In [ ]:
f1_score(y_training, ypred, average=None)

array([0.        , 0.96125654])